# Learning and Planning in Complex Action Spaces $^{2}$
## Proposed method vs. naive PUCB (probabilistic upper confidence tree) bound
## Original Paper authors
* Thomas Hubert $^{1}$
* Julian Schrittwieser $^{1}$
* Ioannis Antonoglou $^{1}$
* Mohammadamin Barekatain $^{1}$
* Simon Schmitt $^{1}$
* David Silver $^{1}$

$^{1}$ DeepMind, London, UK.\
$^{2}$ [PMLR](https://proceedings.mlr.press/v139/), Volume 139, July 2021

## This work was made by
* Manor Zvi, Technion, Israel
* manor.zvi@campus.technion.ac.il

## Code
* Forked from: https://github.com/werner-duvaud/muzero-general.git
* This repo: https://github.com/ManorZ/muzero-general.git
* Branches: 
    * Proposed Method: **continuous**
    * Naive Method: **continuous2**\
    Changes were done solely in the PUCB node expantion, in self_play.py

# Introduction
* In 2016, Deep Mind introduced [AlphaGo](https://www.deepmind.com/research/highlighted-research/alphago)$^{1}$, the first AI to defeat humans at the ancient game of Go.
    * The idea behind AlphaGo was to combine advanced search tree with deep neural networks.
    * One neural network, the “policy network”, selects the next move to play.
    * The other neural network, the “value network”, predicts the winner of the game - the value of each state (commulative discounted sum of future rewards).
    * These deep neural networks are trained by a novel combination of supervised learning from human expert games, and reinforcement learning from games of self-play in two phases: 
        1) It was introduced to numerous amateur games to help it develop an understanding of reasonable human play.
        2) Then it played against different versions of itself until it became the greatest Go player of all time.
* In 2017, they Introduced [AlphaGo-Zero](https://www.deepmind.com/blog/alphago-zero-starting-from-scratch)$^{2}$.
    * While AlphaGo learnt the game by initially playing thousands of matches with amateur and professional players, AlphaGo Zero learnt by playing against itself, starting from completely random play.
    * In other words, they removed to supervised learning training phase, and replaced it with a scheme of reinforcement learning self-plays agaist different versions of itself.
* The year after, its successor - [AlphaZero](https://www.deepmind.com/blog/alphazero-shedding-new-light-on-chess-shogi-and-go)$^{3}$ - learned from scratch to master Go, chess and shogi.
    * AlphaZero taught itself from scratch how to master all the games above, beating a world-champion computer program in each case.
    * Nevertheless, impressive as it was, AlphaZero was still limited to perfect-knowledge, fully-observable games, where the next state is completly predictable given the current state and a chosen action.
    * In other words, AlphaZero is in its essence a lookahead search algorithm, and as such, it relys on being given knowledge of its environment’s dynamics, such as the rules of the game or an accurate simulator.
* Then, in 2020, came [MuZero](https://www.deepmind.com/blog/muzero-mastering-go-chess-shogi-and-atari-without-rules)$^{4}$.\
MuZero masters Go, chess, shogi and **Atari** without need to be told the rules, thanks to its ability to plan winning strategies in unknown environments.
    * It does this by **learning a model of its environment** and combining it with AlphaZero’s powerful lookahead tree search.
    * By combining this model with AlphaZero’s powerful lookahead tree search, MuZero set a new state of the art result on the Atari benchmark, while simultaneously matching the performance of AlphaZero in the classic planning challenges of Go, chess and shogi.
    * Until that point, the best results on Atari are from model-free systems, such as DQN $^{6}$, R2D2 $^{7}$ and Agent57 $^{8}$.\
    As the name suggests, model-free algorithms do not use a learned model and instead estimate what is the best action to take next.
    * Model-based algorithms tries to learn an accurate model of an environment’s dynamics, and then using it for planning.\
    However, the complexity of modelling every aspect of an environment has meant these algorithms are unable to compete in visually rich domains, such as Atari.\
    MuZero overcomes this limitation by modeling only certain aspects that are important to the agent’s decision-making process, rather than the whole environment dynamics.
* In 2021, Deep Mind presented MuZero's evolution - [Sampled MuZero](https://www.furidamu.org/blog/2021/07/18/sampled-muzero--learning-and-planning-in-complex-action-spaces/)$^{5}$
    * This paper claim was that MuZero is limited by the size of its game's action space, and therefore can't handle large scale tasks (such as driving) or continous tasks (such as locomotion control).'
    * According to the claim, MuZero has to evaluate in each stage of its tree all available actions, and therefore fail to advance efficiently in very-wide trees.
    * The authors suggest the modify the tree seach exploration-explitation tradeoff formula (named PUCB) to account the fact that a searies of action are being sampled from a prior destribution, rather than using the on-policy values, generated by the network.
    * Furthermore, In the paper, they related shortly to the naive option of just sampling on-policy, but dimly claimed that this method is unstable numricly, and ignore this approach in the evaluation phase. 
    * In this short work I will compare the suggested PUCB with the naive approach - on-policy sampling during the MCTS - and show that (at least for the workloads I have chosen) there is no much differece between the methods.

$^{1}$ Mastering the game of Go with deep neural networks and tree search. David Silver et al. Nature Volume 529. Jan 2016.\
$^{2}$ Mastering the game of Go without human knowledge. David Silver et al. Nature Volume 550, Oct 2017.\
$^{3}$ A general reinforcement learning algorithm that masters chess, shogi, and Go through self-play. David Silver et al. Science Volume 362. Dec 2018.\
$^{4}$ Mastering Atari, Go, chess and shogi by planning with a learned model. Julian Schrittwieser et al. Nature Volume 588, Dec 2020.\
$^{5}$ Learning and Planning in Complex Action Spaces. Hubert et al. PMLR, July 18, 2021.\
$^{6}$ Human-level control through deep reinforcement learning. Volodymyr Mnih et al. Nature Volume 518. Feb 2015.\
$^{7}$ Recurrent Experience Replay in Distributed Reinforcement Learning. Steven Kapturowski et al. ICLR, May 6, 2019.\
$^{8}$ Agent57: Outperforming the Atari Human Benchmark. Adrià Puigdomènech Badia et al. PMLR, July 13, 2020.


## The way from AlphaGo to MuZero
![SegmentLocal](images/AlphaGo2MuZero.jpg "segment")


## MuZero MCTS (Monte Carlo tree Search)
Starting at the current position in the game, MuZero uses the representation function (h) to map from the observation to an embedding used by the neural network ($s^0$).\
Using the dynamics function (g) and the prediction function (f), MuZero can then consider possible future sequences of actions (a), and choose the best action.
![SegmentLocal](images/MuZeroMCTS.gif "segment")


## MuZero Training
MuZero uses the experience it collects when interacting with the environment to train its neural network.\
This experience includes both observations and rewards from the environment, as well as the results of searches performed when deciding on the best action.
![SegmentLocal](images/MuZeroExperience.gif "segment")\
During training, the model is unrolled alongside the collected experience, at each step predicting the previously saved information:\
* the value function v predicts the sum of observed rewards (u),
* the policy estimate (p) predicts the previous search outcome (π),
* the reward estimate r predicts the last observed reward (u).\
![SegmentLocal](images/MuZeroUnrolling.gif "segment")

## Sampled MuZero
* I will spare you the entire 18 pages of ditilled pleasure in the form of long equations and latin letters. For the whole picture, read the paper.\
* You can go over my slides from '048720 - Robot Learning' Seminar which explains pretty well this topic (I think...).\
you can find it here: **slides/Sampled MuZero - 048720 - Jun19.pptx**
* Also, you can watch Thomas Hubert explains it at ICML 2021 workshop: https://crossminds.ai/video/learning-and-planning-in-complex-action-spaces-614bce4e3c7a224a90902cfb/?vid=614bce4e3c7a224a90902cfb&utm_medium=share

Let $ \pi : S \rightarrow P(A) $ be a policy and $ I\pi : S \rightarrow P(A) $ be an improved policy of $ \pi $:\
$ \forall s \in S , v^{I\pi}(s) >= v^{\pi}(s) $.\
when the action space A is too large, it might only be feasible to compute an improved policy over a small subset of actions.\
Let $\{a_i\}$ be K actions sampled from a proposal distribution $\beta$ and $ \hat{\beta}(a|s) = \frac{1}{K} \cdot \sum_{i}\delta _{a,a_{i}} $ the corresponding empirical distribution which is non-zero only on the sampled actions $\{a_i\}$.\
Let us define the sample-based action-independent policy improvement operator as:\
$ \hat{I}_{\beta}\pi(a|s) = \frac{\hat{\beta}}{\beta}(a|s)f(s,a,\hat{Z}_{\beta}(s)) $\
where $ \hat{Z}_{\beta}(s) $ is a unique state dependent normalising factor defined by $ \forall a \in A, \frac{\hat{\beta}}{\beta}(a|s)f(s,a,\hat{Z}_{\beta}(s)) \ge 0 $ and $ \sum_{a\in A}\frac{\hat{\beta}}{\beta}(a|s)f(s,a,\hat{Z}_{\beta}(s)) = 1 $.

For a given random variable $ X $, we have $ E_{a \sim I\pi}[X|s] = lim_{K\rightarrow\infty}\sum_{a \in A}\hat{I}_{\beta}\pi(a|s)X(s,a)$\
Furthermore, $ \sum_{a \in A}\hat{I}_{\beta}\pi(a|s)X(s,a) $ is approximately normally distributed around $ E_{a \sim I\pi}[X|s] $ as $ K\rightarrow\infty $:\
$ \sum_{a \in A}\hat{I}_{\beta}\pi(a|s)X(s,a) \sim N(E_{a \sim I\pi}[X|s], \frac{\sigma^2}{K}) $\
Where $ \sigma^2 = Var_{a \sim \beta}[\frac{f(s,a,Z(s))}{\beta}X(s,a)|s] $

**Corollary** The sample-based policy improvement operator converges in distribution to the true policy improvement operator: $ lim_{K \rightarrow \infty}\hat{I}_{\beta}\pi = I\pi $

The previous expression computing an estimate of E_{a \sim I\pi}[X|s] using the quantity \hat{I}_{\beta}\pi and the sampled actions $\{a_i\}$ can be used for policy improvement and policy evaluation of the improved policy.\
Policy improvement can be performed by for instance instantiating $ X = -log\pi_{\theta} $, minimising the cross-entropy between \pi_{\theta} and the improved policy $ I\pi $ : $ CE = E_{a \sim I\pi}[-log\pi_{\theta}] $

Concretely, Sampled MuZero apply the sampling procedure described above to the MuZero Algorithm.\
MuZero may be understood as combining an inner process of policy iteration, within its Monte-Carlo tree search, and an outer process, in its overall interactions with the environment.\
within its search tree, MuZero estimates values by explicitly averaging n-step returns samples (policy evaluation) and selects the next node to evaluate and expand by recursively maximising (policy improvement) over the probabilistic upper confidence tree (PUCT) bound.\
$ a^*=argmax_{a}Q(s,a)+c(s)\pi(s|a)\frac{\sqrt{\sum_{b}N(s,b)}}{1+N(s,a)} $

And here comes to intersting part.

The paper says:\
A first approach to extending MuZero’s MCTS search is to search over the sampled actions $\{a_{i}\}$ and keep the PUCT formula unchanged, directly using the probabilities $\pi$ coming from the policy network in the PUCT formula just like in MuZero.\
The search’s visit count distribution $f$ can then be used to construct the sampledbased $\frac{\hat{\beta}}{\beta}f$ to correct for the effect of sampling at policy network training time and acting time...\
Theoretically this procedure is not complicated, but in practice it might lead to unstable results because of the $ f/\beta $ term, especially if $f$ is represented by normalised visit counts which have limited numerical precision.

Instead, the authors propose to search with probabilities $\hat{\pi}_{\beta}(s,a)$ proportional to $\frac{\hat{\beta}}{\beta\pi}(s,a)$, in place of $\pi(s,a)$ in the PUCT formula and directly use the resulting visit count distributions just like in MuZero.

The only modification beyond sampling that needs to be made to MuZero is to use $\hat{\pi}_\beta$ instead of $\pi$ in the PUCT formula.\
The rest of the MuZero algorithm, from estimating the values in the search tree by averaging n-step returns, to acting and training the policy network using the visit count distribution, can proceed unchanged.

In the appendix, the authors provided a massive set of proofs and theorem to support thier proposed modification, but no evaluation have done comparing the naive approach to the proposed modification.\
I thought this is weird, and decided to check it myself.




## Code Modifications
Very few changes to the code was needed. 
Althogh Deep Mind didn't release a formal git for this work, Werner Duvaud created nice community version of MuZero (https://github.com/werner-duvaud/muzero-general.git)'
It even have a Sampled MuZero implementation in a side branch called: continuous.\
I created another branch, called continuous2, where I made my changes.\

The relevant parts are at the ucb_score method inside MCTS class.\
In the branch continuous, they implemented the paper with $\beta=\pi$ and then: $\hat{\pi}_{\beta}=\hat{\beta}/\beta\pi=\hat{\beta}$. And since we are sampling actions from continuous space, there is 0 probability for two identical actions being sampled, even for very large $K$.\
Therefore, $\hat{\beta} \sim Uniform(K) = \frac{1}{K}$.\
The relevant code looks like that:\
![SegmentLocal](images/continuous_node_prior.png "segment")

In the branch continuous2, I modified it a bit, to allow on-policy sampling, as it was for discrete distributions in MuZero:\
![SegmentLocal](images/continuous2_node_prior.png "segment")\
But as one can understand, I had to include a 'node prior', similarly to what was done for discrete action spaces in MuZero.\
node prior is the probability a parent assigns to each one of his children, based on its current policy evaluation.\
While not searching through all children, MuZero always expands all children in each MCTS iteration.\
Here, since we are sampling a single action at each node (and $K$ actions overall in $K$ simulations for each 'real world interaction'), I assigned each expanded node its probability as a prior, to be used in the ucb_scode formula.\
![SegmentLocal](images/continuous2_node_expand.png "segment")

And that's it. As mentioned in the paper, the rest of the code (the visit counting probability, etc.) was not changed.

## Results
* I compared the proposed PUCB method with my naive implementaion on several continuous classic control tasks, based on Mujoco simulator.
* For all, I didn't find any major difference between the authors proposed method and my naive implemetation. 
* In the paper, the authors motivated their suggestion by 'unstability' of the naive method's training process & results, without explicitly explain what they mean by 'unstability'.\
Although I didn't have enough resources to put the results unstability claim under test, the training process stability, in terms of loss running std, looks pretty much ok. Not to say better than the proposed method...
* Duo to time & resources limitation, each experiment was conducted once. For higher validity of the results, one should repeat training for eac hagent multiple times, comparing the reward, loss and episode length as the training process goes on.\
* Training data is located in results/\<env\>/\<experiment\> directory, as TensorBoard events. The authors propsed method under continuous branch, and my naive implementation under continuous2 branch.\
To visualize it, do (command line) tensorboard --logdir \<path\>

### Mujoco Walker
* for the sake of clarity of this notebook (and also because it requires a LOT of copy-paste work), I'm presenting here Mujoco-Walker only. 
* In the results directory one can find other environments and trained agents such as: IDP (Inverted Double Pendulum), IP (Inverted Pendulum), Hopper & Swimmer.
* MuJoco Walker's observation space belongs to $R^{1,1,17}$ and it has 6 degrees of freedom (6 continuous actions). 
#### Total Reward
Orange - authors proposed method.\
Blue - ny naive implementation.\
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--total-reward.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--total-reward-smooth.png "segment")
#### Mean Value
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--mean-value.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--mean-value-smooth.png "segment")
#### Episode Length
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--episode-length.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--episode-length-smooth.png "segment")
#### Total Weighted Loss
Total loss is a linear combintation of entropy loss, policy loss, reward loss and value loss.\
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--total-weighted-loss.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--total-weighted-loss-smooth.png "segment")
#### Entropy Loss
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--entropy-loss.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--entropy-loss-smooth.png "segment")
#### Policy Loss
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--policy-loss.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--policy-loss-smooth.png "segment")
#### Reward Loss
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--reward-loss.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--reward-loss-smooth.png "segment")
#### Value Loss
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--value-loss.png "segment")
![SegmentLocal](images/muzero-cont-vs-orig--mujoco-walker-22-07-25--18-51-32--value-loss-smooth.png "segment")
#### MuZero Walker Authors Proposed PUCB
![SegmentLocal](images/muzero-general-walker-orig.gif "segment")
#### MuZero Walker Naive PUCB
![SegmentLocal](images/muzero-general-walker-new.gif "segment")

### Conclusion:
* In this short work I compared 'Learning and Planning in Complex Action Spaces' suggested PUCB formula with a naive version of it, where I just sampled on-policy and used the policy probability as a prior for exploration during the tree search.
* In order to have more valid conclusion, one has to have much more tests, on more environments and much more repeats.
* Nevertheless, based on the results I presented here (and the result not presented here, but exsits in the repo), it seems that the proposed PUCB method have no major affect on the algorithm performance.